# Healthbar Convolutional Neural Network

We build a cnn to detect the player's health, creating our own Terraria digit image classifier.

In [2]:
import tensorflow as tf
from keras import layers, models

c:\Users\Sam\anaconda3\envs\yolov8_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(11, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\Sam\anaconda3\envs\yolov8_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
import os
import numpy as np
from PIL import Image

def load_images(folder_path):
    images=[]
    labels=[]

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Load image
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('L') # Convert img to grayscale
            img = img.resize((28, 28))
            img_array = np.array(img) / 255.0 # Normalize
            img_array = img_array.reshape(28, 28, 1) # Add channel dimension

            # Extract label from filename (Assumes filename starts with a number)
            if filename.startswith('slash'):
                label = 10 # Assings '/' to be encoded as 10
            else:
                label = int(filename[0])
            
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_augmented_dataset(X_train, y_train, augmentation_factor=10):
    """
    Creates augmented dataset from small training set
    """
    
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1, 
        shear_range=0.1, # Shear transformation
        zoom_range=0.1, # Zoom in/out by up to 10%
        brightness_range=[0.7, 1.3],
        fill_mode='nearest', # Specifies how to fill the pixels after transformations
        horizontal_flip=False, # Don't flip digits horizontally
        vertical_flip=False # Don't flip digits vertically
    )
    
    # Fit the generator
    datagen.fit(X_train)
    
    # Generate augmented images
    augmented_images = []
    augmented_labels = []
    
    # Keep original images
    for img, label in zip(X_train, y_train):
        augmented_images.append(img)
        augmented_labels.append(label)
    
    for img, label in zip(X_train, y_train):
        img_batch = img.reshape(1, 28, 28, 1)
        label_batch = np.array([label])
        
        # Generate augmented versions
        aug_iter = datagen.flow(img_batch, label_batch, batch_size=1)
        
        for i in range(augmentation_factor):
            aug_img, aug_label = next(aug_iter)
            augmented_images.append(aug_img[0])
            augmented_labels.append(aug_label[0])
    
    return np.array(augmented_images), np.array(augmented_labels)

In [6]:
from sklearn.model_selection import train_test_split

# Load data from image folder
folder_path = "C:/Users/Sam/Documents/Comp Sci/Terraria Bot/Terraria-Bot/dataset/Health Numbers/Resized_digits"
X, y = load_images(folder_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train_aug, y_train_aug = create_augmented_dataset(X_train, y_train, augmentation_factor=10)

In [9]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.8571 - loss: 0.5328 - val_accuracy: 0.6875 - val_loss: 0.8569
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8889 - loss: 0.5095 - val_accuracy: 0.8125 - val_loss: 0.5780
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9048 - loss: 0.5055 - val_accuracy: 0.8750 - val_loss: 0.5752
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8413 - loss: 0.4723 - val_accuracy: 0.6875 - val_loss: 0.6994
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8730 - loss: 0.4517 - val_accuracy: 0.8125 - val_loss: 0.6055
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.8889 - loss: 0.4301 - val_accuracy: 0.8125 - val_loss: 0.5750
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9365 - loss: 0.3939 - val_accuracy: 0.7500 - val_loss: 0.5607
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9048 - loss: 0.4031 - val_accuracy: 0.7500 - val_loss: 0.5094


In [11]:
# Save the model for real-time use
model.save("text_classifier.keras")